<a href="https://colab.research.google.com/github/Isafon/ECE528/blob/main/ECE528_ASN3_Q2c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECE528 Lab 3 Q2c - Isa Fontana

## Imports!

In [ ]:
import os, io, zipfile, numpy as np, pandas as pd, tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import matplotlib.pyplot as plt
# --- MUST run this BEFORE importing tensorflow in this session ---
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"   # force tf.keras / tf_keras backend

import tensorflow as tf
print("TF:", tf.__version__)

# Use the legacy tf.keras shim explicitly to avoid any ambiguity
import tf_keras as keras
print("tf_keras module:", keras.__file__)

try:
    import tensorflow_model_optimization as tfmot
except ImportError:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--no-deps",
                           "tensorflow-model-optimization==0.8.0"])
    import tensorflow_model_optimization as tfmot
print("TFMOT:", tfmot.__version__)

np.random.seed(42)
tf.random.set_seed(42)
print("TF version:", tf.__version__)

TF: 2.19.0
tf_keras module: /usr/local/lib/python3.12/dist-packages/tf_keras/__init__.py
TFMOT: 0.8.0
TF version: 2.19.0


## Choose File

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving archive.zip to archive.zip


## Unzip the File

In [ ]:
# Create a working folder
DATA_DIR = "./data_asl"
os.makedirs(DATA_DIR, exist_ok=True)

# If a zip was uploaded, extract it
for fname in uploaded.keys():
    if fname.lower().endswith(".zip"):
        with zipfile.ZipFile(io.BytesIO(uploaded[fname]), 'r') as zf:
            zf.extractall(DATA_DIR)
        print(f"Extracted: {fname} -> {DATA_DIR}")

# Figure out where the CSVs ended up (root or inside DATA_DIR)
candidates = [
    "sign_mnist_train.csv",
    "sign_mnist_test.csv",
    os.path.join(DATA_DIR, "sign_mnist_train.csv"),
    os.path.join(DATA_DIR, "sign_mnist_test.csv"),
]

# Build resolved paths
train_csv, test_csv = None, None
for c in candidates:
    if c.endswith("sign_mnist_train.csv") and os.path.exists(c):
        train_csv = c
    if c.endswith("sign_mnist_test.csv") and os.path.exists(c):
        test_csv = c

assert train_csv and test_csv, "Could not find the CSVs. Re-upload the zip or both CSV files."

print("Train CSV:", train_csv)
print("Test  CSV:", test_csv)

Extracted: archive.zip -> ./data_asl
Train CSV: ./data_asl/sign_mnist_train.csv
Test  CSV: ./data_asl/sign_mnist_test.csv


## Load Data

In [ ]:
# Load CSVs
train_df = pd.read_csv(train_csv)
test_df  = pd.read_csv(test_csv)

# Separate labels and pixels
y_train_raw = train_df.pop('label').values
y_test_raw  = test_df.pop('label').values

x_train = train_df.values.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test  = test_df.values.reshape(-1, 28, 28, 1).astype("float32") / 255.0

# Make labels contiguous (handles “missing J and Z”)
uniq = np.unique(np.concatenate([y_train_raw, y_test_raw]))
remap = {old:i for i, old in enumerate(sorted(uniq))}
y_train = np.array([remap[v] for v in y_train_raw])
y_test  = np.array([remap[v] for v in y_test_raw])

num_classes = len(uniq)  # should be 24
print("Shapes:", x_train.shape, x_test.shape)
print("Classes detected:", num_classes, "Original label ids:", uniq)

Shapes: (27455, 28, 28, 1) (7172, 28, 28, 1)
Classes detected: 24 Original label ids: [ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]


## Model It

In [ ]:
def CBR(filters):
    # Conv -> BatchNorm -> ReLU (BN immediately before activation per instructions)
    return tf.keras.Sequential([
        layers.Conv2D(filters, 3, padding="same", use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),
    ])

inputs = layers.Input((28, 28, 1))
x = CBR(32)(inputs);  x = CBR(32)(x);  x = layers.MaxPool2D()(x);  x = layers.Dropout(0.25)(x)
x = CBR(64)(x);       x = CBR(64)(x);  x = layers.MaxPool2D()(x);  x = layers.Dropout(0.25)(x)
x = CBR(128)(x);      x = layers.Conv2D(128, 3, padding="same", use_bias=False)(x)
x = layers.BatchNormalization()(x); x = layers.ReLU()(x)
x = layers.GlobalAveragePooling2D()(x); x = layers.Dropout(0.40)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs, outputs, name="asl_mnist_cnn")
model.summary()

Model: "asl_mnist_cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 sequential (Sequential)     (None, 28, 28, 32)        416       
                                                                 
 sequential_1 (Sequential)   (None, 28, 28, 32)        9344      
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 14, 14, 32)        0         
                                                                 
 sequential_2 (Sequential)   (None, 14, 14, 64)        18688     
                                                     

## Train It

In [ ]:
from tensorflow.keras import callbacks

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

rlrop = callbacks.ReduceLROnPlateau(
    monitor="val_accuracy", factor=0.5, patience=2, min_lr=1e-5, verbose=1
)
es = callbacks.EarlyStopping(
    monitor="val_accuracy", patience=6, restore_best_weights=True, verbose=1
)

history = model.fit(
    x_train, y_train,
    epochs=60,           # longer run; ES will stop early
    batch_size=128,
    validation_split=0.10,     # from TRAIN only
    callbacks=[rlrop, es],
    verbose=2
)

Epoch 1/60
194/194 - 9s - loss: 1.4295 - accuracy: 0.5942 - val_loss: 8.3793 - val_accuracy: 0.0477 - lr: 0.0010 - 9s/epoch - 47ms/step
Epoch 2/60
194/194 - 1s - loss: 0.2374 - accuracy: 0.9596 - val_loss: 2.1539 - val_accuracy: 0.3343 - lr: 0.0010 - 1s/epoch - 6ms/step
Epoch 3/60
194/194 - 1s - loss: 0.0747 - accuracy: 0.9916 - val_loss: 0.0445 - val_accuracy: 0.9967 - lr: 0.0010 - 1s/epoch - 6ms/step
Epoch 4/60
194/194 - 1s - loss: 0.0329 - accuracy: 0.9972 - val_loss: 0.2259 - val_accuracy: 0.9330 - lr: 0.0010 - 1s/epoch - 6ms/step
Epoch 5/60
194/194 - 1s - loss: 0.0481 - accuracy: 0.9920 - val_loss: 0.0137 - val_accuracy: 0.9989 - lr: 0.0010 - 1s/epoch - 6ms/step
Epoch 6/60
194/194 - 1s - loss: 0.0209 - accuracy: 0.9975 - val_loss: 0.0024 - val_accuracy: 1.0000 - lr: 0.0010 - 1s/epoch - 6ms/step
Epoch 7/60
194/194 - 1s - loss: 0.0168 - accuracy: 0.9976 - val_loss: 0.0048 - val_accuracy: 1.0000 - lr: 0.0010 - 1s/epoch - 6ms/step
Epoch 8/60

Epoch 8: ReduceLROnPlateau reducing learni

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print("Q1 Test accuracy:", round(float(test_acc), 4))

Q1 Test accuracy: 0.9955


## Accuracy Overall (Proof)

In [ ]:
# === Q1: summarize training/validation accuracy over all completed epochs ===
import numpy as np

hist = history.history
# Works with both old/new key names
train_key = 'accuracy' if 'accuracy' in hist else 'acc'
val_key   = 'val_accuracy' if 'val_accuracy' in hist else 'val_acc'

train_acc = np.array(hist[train_key], dtype=float)
val_acc   = np.array(hist[val_key],   dtype=float)

epochs_run = len(val_acc)
best_idx   = int(np.argmax(val_acc))           # 0-based
best_epoch = best_idx + 1                      # 1-based
best_val   = float(val_acc[best_idx])
mean_val   = float(val_acc.mean())
mean_train = float(train_acc.mean())

print(f"Epochs completed: {epochs_run}")
print(f"Mean TRAIN accuracy over epochs: {mean_train:.4f}")
print(f"Mean VAL   accuracy over epochs: {mean_val:.4f}")
print(f"Best VAL accuracy: {best_val:.4f} (epoch {best_epoch})")
print(f"Last VAL accuracy: {val_acc[-1]:.4f}")

try:
    print(f"TEST accuracy: {float(test_acc):.4f}")
except NameError:
    print("TEST accuracy: (run your evaluate cell to show this)")

# Quick pass/fail for the assignment target
meets_target = (mean_val >= 0.92) or ('test_acc' in globals() and float(test_acc) >= 0.92)
print("Meets ≥92% target (mean VAL or TEST):", "Yes" if meets_target else "Not yet")

Epochs completed: 12
Mean TRAIN accuracy over epochs: 0.9606
Mean VAL   accuracy over epochs: 0.8592
Best VAL accuracy: 1.0000 (epoch 6)
Last VAL accuracy: 1.0000
TEST accuracy: 0.9955
Meets ≥92% target (mean VAL or TEST): Yes


## Save the Model

In [ ]:
model.save("asl_mnist_baseline.keras")
print("Saved: asl_mnist_baseline.keras")

Saved: asl_mnist_baseline.keras


# Q2a New Stuff Here

In [ ]:
SAVE_PATH = "asl_mnist_baseline.keras"
if "model" in globals():
    model.save(SAVE_PATH)
    print(f"Saved trained model to {SAVE_PATH}")
else:
    print("No in-memory model found; loading from disk...")
    model = tf.keras.models.load_model(SAVE_PATH)
    print(f"Loaded model from {SAVE_PATH}")

# Sanity: have test tensors?
assert "x_test" in globals() and "y_test" in globals(), "x_test / y_test not found (rerun Q1 data cells)."
print("Test set:", x_test.shape, y_test.shape)

Saved trained model to asl_mnist_baseline.keras
Test set: (7172, 28, 28, 1) (7172,)


## Helper Method

In [ ]:
def tflite_accuracy(tflite_path, x, y, batch_size=128):
    import numpy as np
    import tensorflow as tf
    from tensorflow.lite.python.interpreter import Interpreter  # deprecation warning is fine for now

    x = x.astype(np.float32)
    interp = Interpreter(model_path=tflite_path)
    # We’ll (re)allocate per batch, so no allocate_tensors() yet

    # Read expected spatial dims
    in_det  = interp.get_input_details()[0]
    out_det = interp.get_output_details()[0]
    # Safe default if the model stores a dummy batch dim
    H, W, C = int(in_det['shape'][1]), int(in_det['shape'][2]), int(in_det['shape'][3])

    n = len(x)
    correct = 0
    start = 0
    while start < n:
        end = min(start + batch_size, n)
        batch = x[start:end]

        # ---- KEY: resize to current batch size, then allocate, then run
        interp.resize_tensor_input(in_det['index'], [end - start, H, W, C], strict=False)
        interp.allocate_tensors()
        interp.set_tensor(in_det['index'], batch)
        interp.invoke()
        preds = interp.get_tensor(out_det['index'])   # shape [B, 24]
        correct += int((np.argmax(preds, axis=1) == y[start:end]).sum())
        start = end

    return correct / n

## Convert to float TFLite

In [ ]:
# Float TFLite (baseline)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_float = converter.convert()
open("s_mnist.tflite", "wb").write(tflite_float)

print("Wrote s_mnist.tflite")

Wrote s_mnist.tflite


## Convert with Dynamic Range

In [ ]:
# Dynamic range quantization (weights → int8; activations quantized dynamically at runtime)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_dyn = converter.convert()
open("s_mnist_quant_dyn.tflite", "wb").write(tflite_dyn)

print("Wrote s_mnist_quant_dyn.tflite")

Wrote s_mnist_quant_dyn.tflite


## Evaluate BOTH Models

In [ ]:
acc_float = tflite_accuracy("s_mnist.tflite", x_test, y_test)       # or tflite_accuracy_b1(...)
acc_dyn   = tflite_accuracy("s_mnist_quant_dyn.tflite", x_test, y_test)

import os
mb = lambda p: os.path.getsize(p)/1024/1024
print(f"Float TFLite : acc={acc_float:.4f} | size={mb('s_mnist.tflite'):.2f} MB")
print(f"Dynamic-Range: acc={acc_dyn:.4f} | size={mb('s_mnist_quant_dyn.tflite'):.2f} MB")
print(f"Δacc (dyn - float): {acc_dyn-acc_float:+.4f}")
print(f"Size reduction     : {(1 - mb('s_mnist_quant_dyn.tflite')/mb('s_mnist.tflite'))*100:.1f}%")

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


Float TFLite : acc=0.9955 | size=1.11 MB
Dynamic-Range: acc=0.9960 | size=0.29 MB
Δacc (dyn - float): +0.0004
Size reduction     : 74.0%


# Q2b Starts Here with New Code

In [ ]:
import numpy as np

def rep_ds_16x8():
    # ~200 samples is plenty for MNIST-size models
    for i in range(200):
        yield [x_train[i:i+1].astype(np.float32)]

## Convert to Int16 Activations + Int8 Weights (16x8)

In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = rep_ds_16x8

# Ask for the experimental 16x8 kernels; allow float fallback if any op isn’t supported
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8,
    tf.lite.OpsSet.TFLITE_BUILTINS  # fallback (keeps conversion robust)
]

tflite_16x8 = converter.convert()
open("s_mnist_quant_int16x8.tflite", "wb").write(tflite_16x8)
print("Wrote s_mnist_quant_int16x8.tflite")

Wrote s_mnist_quant_int16x8.tflite


## A TFLite accuracy helper

In [ ]:
import numpy as np
import tensorflow as tf

# Robust TFLite eval: batch=1, no repeated resize/allocate
def tflite_accuracy_b1(tflite_path, x, y):
    import numpy as np
    try:
        # If LiteRT is available, use it; otherwise fall back to TF's Interpreter
        from ai_edge_litert.python.interpreter import Interpreter
    except Exception:
        from tensorflow.lite.python.interpreter import Interpreter

    itp = Interpreter(model_path=tflite_path, num_threads=2)
    itp.allocate_tensors()

    in_det  = itp.get_input_details()[0]
    out_det = itp.get_output_details()[0]
    in_idx   = in_det["index"]
    in_dtype = in_det["dtype"]
    q = in_det.get("quantization_parameters", {})
    scale = float(q.get("scales", [1.0])[0]) if len(q.get("scales", [])) else 1.0
    zero  = int(q.get("zero_points", [0])[0]) if len(q.get("zero_points", [])) else 0

    # one-time resize to (1, H, W, C)
    itp.resize_tensor_input(in_idx, (1,)+tuple(x.shape[1:]), strict=False)
    itp.allocate_tensors()

    correct = 0
    for i in range(len(x)):
        xi = x[i:i+1]
        if in_dtype == np.float32:
            xi = xi.astype(np.float32)
        elif in_dtype in (np.int8, np.uint8, np.int16):
            xi = np.round(xi/scale + zero).astype(in_dtype)
        else:
            raise ValueError(f"Unsupported input dtype: {in_dtype}")

        itp.set_tensor(in_idx, xi)
        itp.invoke()
        pred = itp.get_tensor(out_det["index"]).argmax(axis=-1)[0]
        correct += int(pred == y[i])

    return correct / len(x)

## Evaluate all three TFLite models and compare sizes

In [ ]:
import warnings
warnings.filterwarnings("ignore", message=".*tf.lite.Interpreter is deprecated.*")

In [ ]:
import os
mb = lambda p: os.path.getsize(p)/1024/1024

acc_float = tflite_accuracy_b1("s_mnist.tflite",               x_test, y_test)
acc_dyn   = tflite_accuracy_b1("s_mnist_quant_dyn.tflite",     x_test, y_test)
acc_16x8  = tflite_accuracy_b1("s_mnist_quant_int16x8.tflite", x_test, y_test)

print(f"{'Model':<28} {'Acc':>8}   {'Size (MB)':>9}")
print("-"*50)
print(f"{'Float TFLite':<28} {acc_float:>8.4f}   {mb('s_mnist.tflite'):>9.2f}")
print(f"{'Dynamic-Range (int8W)':<28} {acc_dyn:>8.4f}   {mb('s_mnist_quant_dyn.tflite'):>9.2f}")
print(f"{'Int16x8 (act16, w8)':<28} {acc_16x8:>8.4f}   {mb('s_mnist_quant_int16x8.tflite'):>9.2f}")

print("\nDeltas vs Float:")
print(f"  Dyn acc Δ  : {acc_dyn-acc_float:+.4f} | size ↓ {100*(1-mb('s_mnist_quant_dyn.tflite')/mb('s_mnist.tflite')):>.1f}%")
print(f"  16x8 acc Δ : {acc_16x8-acc_float:+.4f} | size ↓ {100*(1-mb('s_mnist_quant_int16x8.tflite')/mb('s_mnist.tflite')):>.1f}%")

Model                             Acc   Size (MB)
--------------------------------------------------
Float TFLite                   0.9955        1.11
Dynamic-Range (int8W)          0.9960        0.29
Int16x8 (act16, w8)            0.9978        0.30

Deltas vs Float:
  Dyn acc Δ  : +0.0004 | size ↓ 74.0%
  16x8 acc Δ : +0.0022 | size ↓ 73.0%


# Q2c Code Starts Here

In [30]:
# --- Q2c bootstrap: reload what we need without rerunning Q1/Q2a/Q2b ---
import tensorflow as tf, numpy as np, os, pandas as pd

# Load the trained float model from Q1
base_model = tf.keras.models.load_model("asl_mnist_baseline.keras")

if 'x_test' not in globals() or 'y_test' not in globals():
    test = pd.read_csv("sign_mnist_test.csv")
    y_test = test['label'].to_numpy().astype('int64')
    X = test.drop('label', axis=1).to_numpy().reshape(-1, 28, 28, 1)
    x_test = (X / 255.0).astype('float32')
print("Ready for Q2c:", base_model.name, "| test:", x_test.shape, y_test.shape)

for f in ["s_mnist.tflite", "s_mnist_quant_dyn.tflite", "s_mnist_quant_int16x8.tflite"]:
    print(f, "✓" if os.path.exists(f) else "✗ (make in Q2a/Q2b or skip in table)")

Ready for Q2c: asl_mnist_cnn | test: (7172, 28, 28, 1) (7172,)
s_mnist.tflite ✓
s_mnist_quant_dyn.tflite ✓
s_mnist_quant_int16x8.tflite ✓


## Rebuild Q1 Model

In [ ]:
# # Load your trained float model from Q1/Q2a
# BASE_PATH = "asl_mnist_baseline.keras"
# base_float = tf.keras.models.load_model(BASE_PATH)

# def build_asl_seq(input_shape=(28,28,1), num_classes=24):
#     return tf.keras.Sequential([
#         tf.keras.layers.Input(shape=input_shape),

#         tf.keras.layers.Conv2D(32, 3, padding="same", use_bias=False),
#         tf.keras.layers.BatchNormalization(), tf.keras.layers.ReLU(),
#         tf.keras.layers.Conv2D(64, 3, padding="same", use_bias=False),
#         tf.keras.layers.BatchNormalization(), tf.keras.layers.ReLU(),
#         tf.keras.layers.MaxPool2D(), tf.keras.layers.Dropout(0.25),

#         tf.keras.layers.Conv2D(128, 3, padding="same", use_bias=False),
#         tf.keras.layers.BatchNormalization(), tf.keras.layers.ReLU(),

#         tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dropout(0.30),

#         tf.keras.layers.Dense(num_classes, activation="softmax"),
#     ], name="asl_mnist_seq")

# seq_model = build_asl_seq(input_shape=base_float.input_shape[1:],
#                           num_classes=base_float.output_shape[-1])

# # best-effort weight copy (if shapes match, this succeeds)
# try:
#     seq_model.set_weights(base_float.get_weights())
#     print("✓ Copied weights from baseline → Sequential.")
# except Exception as e:
#     print("⚠ Could not copy weights 1:1, will fine-tune more. Reason:", e)

# seq_model.summary()

⚠ Could not copy weights 1:1, will fine-tune more. Reason: You called `set_weights(weights)` on layer "asl_mnist_seq" with a weight list of length 32, but the layer was expecting 17 weights. Provided weights: [array([[[[ 0.02813086,  0.13796699,  0.0631818 , ...
Model: "asl_mnist_seq"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 28, 28, 32)        288       
                                                                 
 batch_normalization_6 (Bat  (None, 28, 28, 32)        128       
 chNormalization)                                                
                                                                 
 re_lu_6 (ReLU)              (None, 28, 28, 32)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 28, 28, 64)        18432     
                                                      

## Flattening the Saved Model

In [31]:
import tensorflow as tf

# 1) Recreate every layer fresh (no graph reuse)
def _clone_layer(layer):
    return layer.__class__.from_config(layer.get_config())

# 2) Recursively inline any nested Models/Sequentials → single Functional graph
def build_flat_functional(model: tf.keras.Model) -> tf.keras.Model:
    x = tf.keras.Input(shape=model.input_shape[1:])
    y = x
    def apply_stack(inp, layers):
        z = inp
        for lyr in layers:
            if isinstance(lyr, tf.keras.layers.InputLayer):
                continue
            if isinstance(lyr, tf.keras.Model):
                z = apply_stack(z, lyr.layers)  # inline submodel
            else:
                z = _clone_layer(lyr)(z)
        return z
    y = apply_stack(x, model.layers)
    return tf.keras.Model(x, y, name=model.name + "_flat")

flat_float = build_flat_functional(base_model)

# 3) Copy weights by name when possible; fallback to by-shape
def copy_weights_by_name_or_shape(src, dst):
    matched = 0
    for d in dst.layers:
        try:
            s = src.get_layer(d.name)
            if s.get_weights() and all(a.shape == b.shape for a,b in zip(s.get_weights(), d.get_weights())):
                d.set_weights(s.get_weights()); matched += 1
        except Exception:
            pass
    if matched == 0:
        # Coarse fallback by shape order
        sw = [w for L in src.layers for w in L.get_weights()]
        di = 0
        for d in dst.layers:
            dw = d.get_weights()
            if not dw: continue
            take = []
            for w in dw:
                while di < len(sw) and sw[di].shape != w.shape:
                    di += 1
                if di == len(sw): break
                take.append(sw[di]); di += 1
            if len(take) == len(dw):
                d.set_weights(take)

copy_weights_by_name_or_shape(base_model, flat_float)

# (Optional) quick float check
flat_float.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
_ = flat_float.evaluate(x_test, y_test, verbose=0)
print("Flattened model ready.")
flat_float.summary()

Flattened model ready.
Model: "asl_mnist_cnn_flat"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 32)        288       
                                                                 
 batch_normalization (Batch  (None, 28, 28, 32)        128       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 28, 28, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9216      
                                                                 
 batch_normalization_1 (Bat  (None, 28, 28, 32)        128       
 chNormalization)        

## Quantization

In [32]:
# If missing, install quietly then import
try:
    import tensorflow_model_optimization as tfmot
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--no-deps",
                           "tensorflow-model-optimization==0.8.0"])
    import tensorflow_model_optimization as tfmot

# Build QAT model from the flattened float model
annotated = tfmot.quantization.keras.quantize_annotate_model(flat_float)
qat_model = tfmot.quantization.keras.quantize_apply(annotated)

qat_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

from tensorflow.keras import callbacks
es_qat = callbacks.EarlyStopping(monitor="val_accuracy", patience=4,
                                 restore_best_weights=True, verbose=1)

history_qat = qat_model.fit(
    # use train split if available; otherwise fine-tune on a val split from test (last resort)
    x_train if 'x_train' in globals() else x_test,
    y_train if 'y_train' in globals() else y_test,
    epochs=8,
    batch_size=128,
    validation_split=0.10,
    callbacks=[es_qat],
    verbose=2
)

qat_loss, qat_acc = qat_model.evaluate(x_test, y_test, verbose=0)
print(f"QAT (fake-quant) test accuracy: {qat_acc:.4f}")

Epoch 1/8
194/194 - 8s - loss: 2.7450 - accuracy: 0.2249 - val_loss: 7.2884 - val_accuracy: 0.0495 - 8s/epoch - 43ms/step
Epoch 2/8
194/194 - 3s - loss: 1.4222 - accuracy: 0.5194 - val_loss: 5.9802 - val_accuracy: 0.0849 - 3s/epoch - 15ms/step
Epoch 3/8
194/194 - 3s - loss: 0.9266 - accuracy: 0.6802 - val_loss: 0.8240 - val_accuracy: 0.7582 - 3s/epoch - 16ms/step
Epoch 4/8
194/194 - 3s - loss: 0.6666 - accuracy: 0.7765 - val_loss: 0.2636 - val_accuracy: 0.9432 - 3s/epoch - 16ms/step
Epoch 5/8
194/194 - 3s - loss: 0.5056 - accuracy: 0.8400 - val_loss: 0.1636 - val_accuracy: 0.9690 - 3s/epoch - 16ms/step
Epoch 6/8
194/194 - 3s - loss: 0.4001 - accuracy: 0.8753 - val_loss: 0.1055 - val_accuracy: 0.9898 - 3s/epoch - 16ms/step
Epoch 7/8
194/194 - 3s - loss: 0.3255 - accuracy: 0.9039 - val_loss: 0.0987 - val_accuracy: 0.9792 - 3s/epoch - 15ms/step
Epoch 8/8
194/194 - 3s - loss: 0.2641 - accuracy: 0.9263 - val_loss: 0.0609 - val_accuracy: 0.9924 - 3s/epoch - 16ms/step
Restoring model weights 

## Convert QAT to fully-integer TFLite (int8 in/out)

In [33]:
import numpy as np, tensorflow as tf

# Use a small calibration set; prefer train if available else test
x_cal = (x_train if 'x_train' in globals() else x_test)[:300].astype('float32')

def representative_dataset():
    for i in range(len(x_cal)):
        yield [x_cal[i:i+1]]

# Self-check the generator shape/dtype
sample = next(iter(representative_dataset()))
print("Rep sample:", [(t.shape, t.dtype) for t in sample])

converter = tf.lite.TFLiteConverter.from_keras_model(qat_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type  = tf.int8
converter.inference_output_type = tf.int8

tflite_qat = converter.convert()
open("s_mnist_quant_aware_training.tflite", "wb").write(tflite_qat)
print("Wrote s_mnist_quant_aware_training.tflite")

Rep sample: [((1, 28, 28, 1), dtype('float32'))]


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Wrote s_mnist_quant_aware_training.tflite


## FINALLY Comparing all FOUR Models

In [34]:
import os, numpy as np, tensorflow as tf

# Helper (batch=1) — define if you don’t already have it
def tflite_accuracy_b1(tflite_path, x, y):
    inter = tf.lite.Interpreter(model_path=tflite_path); inter.allocate_tensors()
    inp = inter.get_input_details()[0]; out = inter.get_output_details()[0]
    scale, zero = inp.get("quantization", (1.0, 0))
    correct = 0
    for i in range(len(x)):
        xb = x[i:i+1]
        if inp["dtype"] == np.float32:
            feed = xb.astype(np.float32)
        elif inp["dtype"] == np.int8:
            feed = np.round(xb/scale + zero).astype(np.int8)
        elif inp["dtype"] == np.int16:
            feed = np.round(xb/scale + zero).astype(np.int16)
        else:
            raise TypeError(f"Unsupported input dtype: {inp['dtype']}")
        inter.set_tensor(inp["index"], feed)
        inter.invoke()
        pred = inter.get_tensor(out["index"]).argmax(axis=1)[0]
        correct += int(pred == y[i])
    return correct / len(x)

mb = lambda p: os.path.getsize(p)/1024/1024

# Some files come from Q2a/Q2b; skip gracefully if missing
models = [
    ("Float TFLite",               "s_mnist.tflite"),
    ("Dynamic-Range (int8 weights)","s_mnist_quant_dyn.tflite"),
    ("Int16x8 (act16, w8)",        "s_mnist_quant_int16x8.tflite"),
    ("QAT full-int8 (in/out)",     "s_mnist_quant_aware_training.tflite"),
]

print(f"{'Model':<30} {'Acc':>8} {'Size (MB)':>12}")
print("-"*54)
for name, path in models:
    if not os.path.exists(path):
        print(f"{name:<30} {'(missing)':>8} {'--':>12}")
        continue
    acc = tflite_accuracy_b1(path, x_test, y_test)
    print(f"{name:<30} {acc:>8.4f} {mb(path):>12.2f}")

Model                               Acc    Size (MB)
------------------------------------------------------
Float TFLite                     0.9955         1.11
Dynamic-Range (int8 weights)     0.9960         0.29
Int16x8 (act16, w8)              0.9978         0.30
QAT full-int8 (in/out)           0.9555         0.29


## Isa's Write Up

#### For setup, I reused the ASL baseline CNN from Q1 and exported three TensorFlow Lite variants using post-training quantization (PTQ): (1) Float TFLite (no quantization), (2) Dynamic-range (int8 weights, float I/O), and (3) Int16x8 (int16 activations + int8 weights). Then I performed Quantization-Aware Training (QAT) on a flattened copy of the same architecture (BatchNorm -> activation order preserved), fine-tuned for a few epochs with a small learning rate, and converted the QAT model to a fully-integer TFLite model (int8 in/out) using a representative dataset for calibration.

## Results!

####

*   Float TFLite — Acc 0.9955, 1.11 MB
*   Dynamic-range (int8W) — Acc 0.9960, 0.29 MB (−74%)
*   16x8 (act16, int8W) — Acc 0.9978, 0.30 MB (−73%)
*   QAT full-int8 (in/out) - Acc 0.9555, 0.29 MB




## My Discussion

#### Isa's Observations:

*   PTQ can be very strong when the model is well-behaved: The dynamic-range model matches float accuracy (slightly higher within noise) and shrinks size by ~74%. This makes sense: only weights are statically quantized to int8; activations are still effectively handled in float (or quantized on-the-fly by dynamic kernels), so quantization error is limited.

*   Int16x8 often recovers the “hard” activation detail:
Using 16-bit activations preserves more dynamic range in intermediate tensors, which typically helps vision models with BatchNorm+ReLU blocks. Here it achieves the best accuracy (0.9978) with ~73% size reduction relative to float. This is a common pattern when activations are the sensitive part of the network.

*   What made QAT underperform here (0.9555)?... So sad


1.   Conversion target mismatch: QAT trains with fake-quant nodes assuming a specific quantization scheme. If the final TFLite export enforces full integer I/O (int8 in/out) and performs calibration with a small or distribution-mismatched representative set, you can get quantization points that don't match training statistics. That gap shows up as an accuracy drop after conversion, even if the Keras QAT model (“fake-quant” evaluation) is very high (so sad).

2.   Representative dataset sensitivity: QAT still needs accurate calibration at export time. If the calibration subset is too small, unbalanced across classes, or not preprocessed identically (normalization / dtype) to training, the scales/zero-points can be off.

3.   Model/graph changes during export: Folding BatchNorm into Conv and per-tensor vs. per-channel weight quantization can differ between the QAT graph and the converter's kernels. Those implementation details can matter on small 28x28 inputs.

4. Overfitting during short fine-tune: QAT fine-tunes added fake-quant constraints; with very few epochs it's possible to overfit the training/val split yet be brittle after integer export.